# Microsoft Defender for Endpoint (MDE) (previously MDATP) REST API FUN

- DOCUMENTATION on an introduction to the MDE/MDATP REST API: [here!](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/apis-intro)
- BLOG on Automating Security Operations Using Windows Defender ATP APIs with Python and Jupyter Notebooks: [here!](https://techcommunity.microsoft.com/t5/microsoft-defender-for-endpoint/automating-security-operations-using-windows-defender-atp-apis/ba-p/294434)
- DOCUMENTATION on the list of exposed MDE/MDATP APIs: [here!](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/exposed-apis-list)
    - NOTE: there are multiple versions of the API!

|#|Topic|Description|Link|
|:-|:-|:-|:-|
|1|Advanced Hunting|Run queries from API.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/run-advanced-query-api)|
|2|Alerts|Run API calls such as get alerts, create alert, update alert and more.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/alerts)|
|3|Domains|Run API calls such as get domain-related devices, domain statistics and more.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/get-domain-related-alerts)|
|4|Files|Run API calls such as get file information, file related alerts, file related devices, and file statistics.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/files)|
|5|IPs|Run API calls such as get IP-related alerts and get IP statistics.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/get-ip-related-alerts)|
|6|Machines|Run API calls such as get devices, get devices by ID, information about logged on users, edit tags and more.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/machine)|
|7|Machine Actions|Run API call such as Isolation, Run anti-virus scan and more.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/machineaction)|
|8|Indicators|Run API call such as create Indicator, get Indicators and delete Indicators.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/ti-indicator)|
|9|Users|Run API calls such as get user-related alerts and user-related devices.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/user)|
|10|Score|Run API calls such as get exposure score or get device secure score.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/score)|
|11|Software|Run API calls such as list vulnerabilities by software.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/software)|
|12|Vulnerability|Run API calls such as list devices by vulnerability.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/vulnerability)|
|13|Recommendation|Run API calls such as Get recommendation by ID.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/recommendation)|

Each one of those exposed APIs has multiple functions!

## Requirements for accessing a MDE via REST API

To access the MDE of someone (ex. a customer) you need to:
1. Create an App in the Azure AD. An easy documentation [HERE!](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/exposed-apis-create-app-partners)
After setting-up the App, check in the overview the following info: (1) Application (client) ID, (2) Directory (tenant) ID, and (3) Secret value

3. VERY IMPORTANT. The global admin should give you consent to access the App (client_id): $https://login.microsoftonline.com/common/oauth2/authorize?prompt=consent&client_id=<HERE-THE-CLIENT-ID>&response_type=code&sso_reload=true$

## Python libraries used

In [2]:
import pandas as pd
import json
import urllib.parse

<h1 align='center'>=====================================<br>VERY IMPORTANT<br>ALL FUNCTIONS DEPENDS ON THIS ACCESS TOKEN!!</h1>

## FUNCTION mde_getaadtoken()

In [10]:
def mde_getaadtoken(tenantId, appId, appSecret):
    url = "https://login.windows.net/%s/oauth2/token" % (tenantId)
    resourceAppIdUri = 'https://api.securitycenter.windows.com'
    body = {
        'resource' : resourceAppIdUri,
        'client_id' : appId,
        'client_secret' : appSecret,
        'grant_type' : 'client_credentials'
    }
    data = urllib.parse.urlencode(body).encode("utf-8")
    req = urllib.request.Request(url, data)
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    aadToken = jsonResponse["access_token"]
    return aadToken

In [11]:
access_token = mde_getaadtoken(tenantId, appId, appSecret)

<h1 align='center'>=====================================<br>ADVANCED HUNTING API<br>https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/run-advanced-query-api</h1>

Very nice list of KQL's here: https://github.com/microsoft/Microsoft-365-Defender-Hunting-Queries

In [7]:
def mde_advancedquery_run(access_token,kql_query):
    url = "https://api.securitycenter.microsoft.com/api/advancedqueries/run"
    req = urllib.request.Request(url, 
                                 headers={'Authorization' : "Bearer " + access_token, 'Content-Type':'application/json'},
                                 data=bytes(json.dumps({"Query": kql_query}), encoding="utf-8"))
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['Results'])

<h1 align='center'>=====================================<br>ALERT API<br>https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/alerts</h1>

|Method	| Return Type |Description  |
|:- |:-|:-|
|Get alert|	Alert|	Get a single alert object.|
|List alerts|	Alert| collection	List alert collection.|
|Update alert|	Alert|	Update specific alert.|
|Create alert|	Alert|	Create an alert based on event data obtained from Advanced Hunting.|
|List related domains|	Domain collection|	List URLs associated with the alert.|
|List related files|	File collection|	List the file entities that are associated with the alert.|
|List related IPs|	IP collection|	List IPs that are associated with the alert.|
|Get related machines|	Machine|	The machine that is associated with the alert.|
|Get related users|	User|	The user that is associated with the alert.|

## FUNCTION Get alert

In [4]:
def mde_alerts_getalert(access_token,alert_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alert_id)
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_dict(jsonResponse,orient='index').transpose()

## FUNCTION List alerts

In [13]:
def mde_alerts_listalerts(access_token):
    url = "https://api.securitycenter.windows.com/api/alerts"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

## FUNCTION Update alert

## FUNCTION Create alert

## FUNCTION List related domains

In [61]:
def mde_alerts_listrelateddomains(access_token,alarm_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alarm_id)+"/domains"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

## FUNCTION List related files

In [63]:
def mde_alerts_listrelatedfiles(access_token,alarm_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alarm_id)+"/files"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

## FUNCTION List related IPs

In [59]:
def mde_alerts_listrelatedips(access_token,alarm_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alarm_id)+"/ips"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

## FUNCTION Get related machines

In [56]:
def mde_alerts_getrelatedmachines(access_token,alarm_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alarm_id)+"/machine"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_dict(jsonResponse,orient='index').transpose()

## FUNCTION Get related users

<h1 align='center'>=====================================<br>MACHINE API<br>https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/machine</h1>

|Method|	Return Type|	Description|
|:- |:-|:-|
|List machines|	machine collection|	List set of machine entities in the org.|
|Get machine|	machine	|Get a machine by its identity.|
|Get logged on users|	user collection|	Get the set of User that logged on to the machine.|
|Get related alerts	|alert collection|	Get the set of alert entities that were raised on the machine.|
|Get installed software|	software collection|	Retrieves a collection of installed software related to a given machine ID.|
|Get discovered vulnerabilities|	vulnerability collection|	Retrieves a collection of discovered vulnerabilities related to a given machine ID.|
|Get security recommendations|	recommendation collection|	Retrieves a collection of security recommendations related to a given machine ID.|
|Add or Remove machine tags|	machine|	Add or Remove tag to a specific machine.|
|Find machines by IP|	machine collection|	Find machines seen with IP.|
|Find machines by tag|	machine collection|	Find machines by Tag.|
|Get missing KBs|	KB collection|	Get a list of missing KBs associated with the machine ID.|
|Set device value|	machine collection|	Set the value of a device.|

## FUNCTION List machines

In [69]:
def mde_machines_listmachines(access_token):
    url = "https://api.securitycenter.microsoft.com/api/machines"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

## FUNCTION Get machines

In [77]:
def mde_machines_getmachine(access_token,machine_id):
    url = "https://api.securitycenter.windows.com/api/machines/"+str(machine_id)
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_dict(jsonResponse,orient='index').transpose()